# Orders

✏️ **Exercise**

Today, we will investigate the **orders**, and their associated **review score**.

👉 Our goal is to create a DataFrame with the following features:


| feature_name 	| type 	| description 	|
|:---	|:---:	|:---	|
| `order_id` 	| str 	| the id of the order 	|
| `wait_time` 	| float 	| the number of days between order_purchase_timestamp and order_delivered_customer_date 	|
| `expected_wait_time` 	| float 	| the number of days between order_purchase_timestamp and estimated_delivery_date 	|
| `delay_vs_expected` 	| float 	| if the actual order_delivered_customer_date is later than the estimated delivery date, returns the number of days between the two dates, otherwise return 0 	|
| `order_status` 	| str 	| the status of the order 	|
| `dim_is_five_star` 	| int 	| 1 if the order received a five-star review, 0 otherwise 	|
| `dim_is_one_star` 	| int 	| 1 if the order received a one_star, 0 otherwise 	|
| `review_score` 	| int 	| from 1 to 5 	|
| `number_of_items` 	| int 	| number of items that the order contains 	|
| `number_of_sellers` 	| int 	| number of sellers involved in the order 	|
| `price` 	| float 	| total price of the order paid by customer 	|
| `freight_value` 	| float 	| value of the freight paid by customer 	|
| `distance_customer_seller` 	| float 	| the distance in km between customer and seller (optional) 	|  
  
⚠️ We also want to filter out "non-delivered" orders, unless explicitly specified, otherwise we cannot compute the potential delays.

❓ **Your challenge**: 

- Implement each feature as a separate method within the `Order` class available at `olist/order.py`
- Then, create a method `get_training_data()` that returns the complete DataFrame **without `NaN`s**.

💡 Suggested methodology:
- Use the notebook below to write and test your code step-by-step first
- Then copy the code into `order.py` once you are certain of your code logic
- Focus on the data manipulation logic now, we will analyse the dataset visually in the next challenges

🔥 Notebook best practices (must-read) 👇

<details>
    <summary>▸ <i>click here</i></summary>

From now on, exploratory notebooks are going to become pretty long, and we strongly advise you to follow these notebook principles:
- Code your logic so that your Notebook can always be ran from top to bottom without crashing (Cell --> Run All)
- Name your variables carefully 
- Use dummy names such as `tmp` or `_` for intermediary steps when you know you won't need them for long
- Clear your code and merge cells when relevant to minimize Notebook size (`Shift-M`)
- Hide your cell output if you don't need to see it anymore (double-click on the red `Out[]:` section to the left of your cell).
- Make heavy use of jupyter nbextention `Collapsible Headings` and `Table of Content` (call a TA if you can't find them)
- Use the following shortcuts 
    - `a` to insert a cell above
    - `b` to insert a cell below
    - `dd` to delete a cell
    - `esc` and `arrows` to move between cells
    - `Shift-Enter` to execute cell and move focus to the next one
    - use `Shift + Tab` when you are between method brackets e.g. `groupby()` to get the docs! Repeat a few times to open it permanently

</details>





In [38]:
# Auto reload imported module every time a jupyter cell is executed (handy for olist.order.py updates)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
# Import usual modules
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [40]:
%matplotlib inline

In [41]:
# Import olist data
from olist.data import Olist
olist = Olist()
data = olist.get_data()

Constructed CSV Path: /home/vinodha/code/vinodhaminnotra/04-Decision-Science/01-Project-Setup/data-context-and-setup/data/csv


In [42]:
# What datasets do we have access to now ?
data.keys()

dict_keys(['geolocation', 'products', 'order_payments', 'orders', 'order_items', 'product_category_name_translation', 'order_reviews', 'customers', 'sellers'])

In [43]:
orders = data['orders'].copy() # good practice to be sure not to modify your `data` variable

assert(orders.shape == (99441, 8))

## 1. Code `order.py`

### a) `get_wait_time`
    ❓ Return a Dataframe with:
           order_id, wait_time, expected_wait_time, delay_vs_expected, order_status


🎁 We give you the pseudo-code below 👇 for this first method:

> 1. Inspect the `orders` dataframe
2. Filter the dataframe on `delivered orders`
3. Handle `datetime`
    - Take time to understand what python [`datetime`](https://docs.python.org/3/library/datetime.html) objects are
    - and convert dates from "string" type to "pandas.datetime' using [`pandas.to_datetime()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html)
4. Compute `wait_time` in days as a decimal number (starting from `order_purchase_timestamp`)
5. Compute `expected_wait_time` in days as a decimal number (starting from `order_purchase_timestamp`)
6. Compute `delay_vs_expected` in days as a decimal number (put `0` if the order is delivered earlier than estimated)
7. Check the new dataframe 
8. Once you are satisfied with your code, you can carefully copy-paste it from the notebook to to `olist/order.py`
9. Now try to load the data using your method in `orders.py`
10. Your code will probably not work straightaway
11. Make the necessary changes to make it work from inside a `.py` file

We want `wait_time`, `expected_wait_time` and `delay_vs_expected` in decimal numbers, not as integers. Think about it: a wait time of 8.1 or 8.9 days is a big difference. So you can't use Pandas' `dt.day` because that rounds both down to 8.

<details>
    <summary>💡Hint</summary>

For both `wait_time` and `delay_vs_expected`, you need to subtract the relevant dates/timestamps to get the time difference between the `pandas.datetime` objects. Then, you can either use [`datetime.timedelta()`](https://docs.python.org/3/library/datetime.html#timedelta-objects) or [`np.timedelta64()`](https://numpy.org/doc/stable/reference/arrays.datetime.html#datetime-and-timedelta-arithmetic) to find out how many days that subtraction represents!

</details>

In [44]:
orders = data['orders'].copy()

In [45]:
print(orders.shape)
print(orders.columns)

(99441, 8)
Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date'],
      dtype='object')


In [46]:
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [47]:
delivered_orders = orders[orders['order_status'] == 'delivered'].copy()
print(delivered_orders.shape)
delivered_orders.head()

(96478, 8)


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [48]:
delivered_orders['order_purchase_timestamp'] = pd.to_datetime(delivered_orders['order_purchase_timestamp'])
delivered_orders['order_delivered_customer_date'] = pd.to_datetime(delivered_orders['order_delivered_customer_date'])
delivered_orders['order_estimated_delivery_date'] = pd.to_datetime(delivered_orders['order_estimated_delivery_date'])

In [49]:
print(delivered_orders.dtypes)

order_id                                 object
customer_id                              object
order_status                             object
order_purchase_timestamp         datetime64[ns]
order_approved_at                        object
order_delivered_carrier_date             object
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
dtype: object


In [50]:
delivered_orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26


In [51]:
# Calculate wait_time
delivered_orders['wait_time'] = (delivered_orders['order_delivered_customer_date'] - 
                                 delivered_orders['order_purchase_timestamp']).dt.days

# Check the first few rows to verify the calculation
delivered_orders[['order_id', 'order_purchase_timestamp', 'order_delivered_customer_date', 'wait_time']].head()


,order_id,order_purchase_timestamp,order_delivered_customer_date,wait_time
0,e481f51cbdc54678b7cc49136f2d6af7,2017-10-02 10:56:33,2017-10-10 21:25:13,8.0
1,53cdb2fc8bc7dce0b6741e2150273451,2018-07-24 20:41:37,2018-08-07 15:27:45,13.0
2,47770eb9100c2d0c44946d9cf07ec65d,2018-08-08 08:38:49,2018-08-17 18:06:29,9.0
3,949d5b44dbf5de918fe9c16f97b45f8a,2017-11-18 19:28:06,2017-12-02 00:28:42,13.0
4,ad21c59c0840e6cb83a9ceb5573f8159,2018-02-13 21:18:39,2018-02-16 18:17:02,2.0


In [52]:
# Calculate expected_wait_time
delivered_orders['expected_wait_time'] = (delivered_orders['order_estimated_delivery_date'] - 
                                          delivered_orders['order_purchase_timestamp']).dt.days

# Check the first few rows to verify the calculation
delivered_orders[['order_id', 'order_purchase_timestamp', 'order_estimated_delivery_date', 'expected_wait_time']].head()


,order_id,order_purchase_timestamp,order_estimated_delivery_date,expected_wait_time
0,e481f51cbdc54678b7cc49136f2d6af7,2017-10-02 10:56:33,2017-10-18,15
1,53cdb2fc8bc7dce0b6741e2150273451,2018-07-24 20:41:37,2018-08-13,19
2,47770eb9100c2d0c44946d9cf07ec65d,2018-08-08 08:38:49,2018-09-04,26
3,949d5b44dbf5de918fe9c16f97b45f8a,2017-11-18 19:28:06,2017-12-15,26
4,ad21c59c0840e6cb83a9ceb5573f8159,2018-02-13 21:18:39,2018-02-26,12


In [53]:
# Calculate delay_vs_expected
delivered_orders['delay_vs_expected'] = (
    delivered_orders['order_delivered_customer_date'] - 
    delivered_orders['order_estimated_delivery_date']
).dt.days

# Set negative delays to 0
delivered_orders['delay_vs_expected'] = delivered_orders['delay_vs_expected'].apply(lambda x: max(0, x))

# Check the first few rows to verify the calculation
delivered_orders[['order_id', 'order_delivered_customer_date', 'order_estimated_delivery_date', 'delay_vs_expected']].head()


,order_id,order_delivered_customer_date,order_estimated_delivery_date,delay_vs_expected
0,e481f51cbdc54678b7cc49136f2d6af7,2017-10-10 21:25:13,2017-10-18,0.0
1,53cdb2fc8bc7dce0b6741e2150273451,2018-08-07 15:27:45,2018-08-13,0.0
2,47770eb9100c2d0c44946d9cf07ec65d,2018-08-17 18:06:29,2018-09-04,0.0
3,949d5b44dbf5de918fe9c16f97b45f8a,2017-12-02 00:28:42,2017-12-15,0.0
4,ad21c59c0840e6cb83a9ceb5573f8159,2018-02-16 18:17:02,2018-02-26,0.0


In [54]:
# Select and return the final DataFrame
get_wait_time_df = delivered_orders[['order_id', 'order_status', 'wait_time', 'expected_wait_time', 'delay_vs_expected']]

# Check the final DataFrame
print(wait_time_df.shape)
get_wait_time_df.head()


NameError: name 'wait_time_df' is not defined

👀 Check the dataframe you've just created. <br/> 

💪 When your code works, bring your code over to `olist/order.py` <br/>

🧪 Now, test it by running the following cell 👇 

In [55]:
# Test your code here
from olist.order import Order
Order().get_wait_time()

Constructed CSV Path: /home/vinodha/code/vinodhaminnotra/04-Decision-Science/01-Project-Setup/data-context-and-setup/data/csv


,order_id,order_status,wait_time,expected_wait_time,delay_vs_expected
0,e481f51cbdc54678b7cc49136f2d6af7,delivered,8.0,15,0.0
1,53cdb2fc8bc7dce0b6741e2150273451,delivered,13.0,19,0.0
2,47770eb9100c2d0c44946d9cf07ec65d,delivered,9.0,26,0.0
3,949d5b44dbf5de918fe9c16f97b45f8a,delivered,13.0,26,0.0
4,ad21c59c0840e6cb83a9ceb5573f8159,delivered,2.0,12,0.0
...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,delivered,8.0,18,0.0
99437,63943bddc261676b46f01ca7ac2f7bd8,delivered,22.0,23,0.0
99438,83c1379a015df1e13d02aae0204711ab,delivered,24.0,30,0.0
99439,11c177c8e97725db2631073c19f07b62,delivered,17.0,37,0.0


In [56]:
from nbresult import ChallengeResult
test = Order().get_wait_time()
result = ChallengeResult('wait_time', dve_type=test["delay_vs_expected"].dtype, shape=test.shape, dve_min=test["delay_vs_expected"].min(), dve_max=test["delay_vs_expected"].max())
result.write(); print(result.check())

Constructed CSV Path: /home/vinodha/code/vinodhaminnotra/04-Decision-Science/01-Project-Setup/data-context-and-setup/data/csv

============================= test session starts ==============================
platform linux -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /home/vinodha/.pyenv/versions/3.10.6/envs/lewagon/bin/python
cachedir: .pytest_cache
rootdir: /home/vinodha/code/vinodhaminnotra/04-Decision-Science/02-Statistical-Inference/data-orders, configfile: pytest.ini
plugins: asyncio-0.19.0, typeguard-2.13.3, anyio-3.6.2
asyncio: mode=strict
collecting ... collected 1 item

test_wait_time.py::TestWaitTime::test_wait_time PASSED                   [100%]

============================== 1 passed in 0.13s ===============================


💯 You can commit your code:

git add tests/wait_time.pickle

git commit -m 'Completed wait_time step'

git push origin master



### b) `get_review_score`
     ❓ Returns a DataFrame with:
        order_id, dim_is_five_star, dim_is_one_star, review_score

dim_is_$N$_star should contain `1` if review_score=$N$ and `0` otherwise 

<details>
    <summary markdown='span'>Hints</summary>

Think about `Series.map()` or `DataFrame.apply()`
    
</details>

👉 We load the `reviews` for you

In [57]:
reviews = data['order_reviews'].copy()
assert(reviews.shape == (99224,7))
reviews

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53
...,...,...,...,...,...,...,...
99219,574ed12dd733e5fa530cfd4bbf39d7c9,2a8c23fee101d4d5662fa670396eb8da,5,NaN,NaN,2018-07-07 00:00:00,2018-07-14 17:18:30
99220,f3897127253a9592a73be9bdfdf4ed7a,22ec9f0669f784db00fa86d035cf8602,5,NaN,NaN,2017-12-09 00:00:00,2017-12-11 20:06:42
99221,b3de70c89b1510c4cd3d0649fd302472,55d4004744368f5571d1f590031933e4,5,NaN,"Excelente mochila, entrega super rápida. Super...",2018-03-22 00:00:00,2018-03-23 09:10:43
99222,1adeb9d84d72fe4e337617733eb85149,7725825d039fc1f0ceb7635e3f7d9206,4,NaN,NaN,2018-07-01 00:00:00,2018-07-02 12:59:13


In [58]:
from olist.data import Olist
data = Olist().get_data()
reviews = data['order_reviews'].copy()

Constructed CSV Path: /home/vinodha/code/vinodhaminnotra/04-Decision-Science/01-Project-Setup/data-context-and-setup/data/csv


In [59]:
reviews['dim_is_five_star'] = reviews['review_score'].apply(lambda x: 1 if x == 5 else 0)
reviews['dim_is_one_star'] = reviews['review_score'].apply(lambda x: 1 if x == 1 else 0)

In [60]:
get_review_score_df = reviews[['order_id', 'dim_is_five_star', 'dim_is_one_star', 'review_score']]
print(get_review_score_df.head())

                           order_id  dim_is_five_star  dim_is_one_star  \
0  73fc7af87114b39712e6da79b0a377eb                 0                0   
1  a548910a1c6147796b98fdf73dbeba33                 1                0   
2  f9e4b658b201a9f2ecdecbb34bed034b                 1                0   
3  658677c97b385a9be170737859d3511b                 1                0   
4  8e6bfb81e283fa7e4f11123a3fb894f1                 1                0   

   review_score  
0             4  
1             5  
2             5  
3             5  
4             5  


Once again, 

👀 Check the dataframe you've just created. <br/> 

💪 When your code works, bring your code over to `olist/order.py` <br/>

🧪 Now, test it by running the following cell 👇 

In [61]:
# Test your code here
from olist.order import Order
Order().get_review_score()

Constructed CSV Path: /home/vinodha/code/vinodhaminnotra/04-Decision-Science/01-Project-Setup/data-context-and-setup/data/csv


,order_id,dim_is_five_star,dim_is_one_star,review_score
0,73fc7af87114b39712e6da79b0a377eb,0,0,4
1,a548910a1c6147796b98fdf73dbeba33,1,0,5
2,f9e4b658b201a9f2ecdecbb34bed034b,1,0,5
3,658677c97b385a9be170737859d3511b,1,0,5
4,8e6bfb81e283fa7e4f11123a3fb894f1,1,0,5
...,...,...,...,...
99219,2a8c23fee101d4d5662fa670396eb8da,1,0,5
99220,22ec9f0669f784db00fa86d035cf8602,1,0,5
99221,55d4004744368f5571d1f590031933e4,1,0,5
99222,7725825d039fc1f0ceb7635e3f7d9206,0,0,4


In [62]:
from nbresult import ChallengeResult
result = ChallengeResult('review_score', shape=Order().get_review_score().shape)
result.write(); print(result.check())

Constructed CSV Path: /home/vinodha/code/vinodhaminnotra/04-Decision-Science/01-Project-Setup/data-context-and-setup/data/csv

============================= test session starts ==============================
platform linux -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /home/vinodha/.pyenv/versions/3.10.6/envs/lewagon/bin/python
cachedir: .pytest_cache
rootdir: /home/vinodha/code/vinodhaminnotra/04-Decision-Science/02-Statistical-Inference/data-orders, configfile: pytest.ini
plugins: asyncio-0.19.0, typeguard-2.13.3, anyio-3.6.2
asyncio: mode=strict
collecting ... collected 1 item

test_review_score.py::TestReviewScore::test_review_score PASSED          [100%]

============================== 1 passed in 0.01s ===============================


💯 You can commit your code:

git add tests/review_score.pickle

git commit -m 'Completed review_score step'

git push origin master



### c) `get_number_items`:
     ❓ Returns a DataFrame with:
        order_id, number_of_items (total number of items per order)

In [63]:
order_items = data['order_items'].copy()
get_number_items_df = order_items.groupby('order_id').size().reset_index(name='number_of_items')
print(get_number_items_df.head())

                           order_id  number_of_items
0  00010242fe8c5a6d1ba2dd792cb16214                1
1  00018f77f2f0320c557190d7a144bdd3                1
2  000229ec398224ef6ca0657da4fc703e                1
3  00024acbcdf0a6daa1e931b038114c75                1
4  00042b26cf59d7ce69dfabb4e55b4fd9                1


🧪 Same routine: 
* check your dataframe, 
* bring your code over to `olist/order.py`
* and check that it truly works.

In [64]:
from nbresult import ChallengeResult
result = ChallengeResult('number_items',
                         shape=Order().get_number_items().shape,
                         columns=Order().get_number_items().columns)
result.write(); print(result.check())

Constructed CSV Path: /home/vinodha/code/vinodhaminnotra/04-Decision-Science/01-Project-Setup/data-context-and-setup/data/csv
Constructed CSV Path: /home/vinodha/code/vinodhaminnotra/04-Decision-Science/01-Project-Setup/data-context-and-setup/data/csv

============================= test session starts ==============================
platform linux -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /home/vinodha/.pyenv/versions/3.10.6/envs/lewagon/bin/python
cachedir: .pytest_cache
rootdir: /home/vinodha/code/vinodhaminnotra/04-Decision-Science/02-Statistical-Inference/data-orders, configfile: pytest.ini
plugins: asyncio-0.19.0, typeguard-2.13.3, anyio-3.6.2
asyncio: mode=strict
collecting ... collected 2 items

test_number_items.py::TestNumberItems::test_column_names PASSED          [ 50%]
test_number_items.py::TestNumberItems::test_number_items PASSED          [100%]

============================== 2 passed in 0.36s ===============================


💯 You can commit your code:

git add te

### d) `get_number_sellers`:
     ❓ Returns a DataFrame with:
        order_id, number_of_sellers (total number of unique sellers per order)
        
<details>
    <summary>▸ <i>Hint</i></summary>

`pd.Series.nunique()`
</details>

In [65]:
order_items = data['order_items'].copy()

# Group by order_id and count the number of unique sellers per order
get_number_sellers_df = order_items.groupby('order_id')['seller_id'].nunique().reset_index(name='number_of_sellers')

# Display the resulting DataFrame
print(get_number_sellers_df.head())

                           order_id  number_of_sellers
0  00010242fe8c5a6d1ba2dd792cb16214                  1
1  00018f77f2f0320c557190d7a144bdd3                  1
2  000229ec398224ef6ca0657da4fc703e                  1
3  00024acbcdf0a6daa1e931b038114c75                  1
4  00042b26cf59d7ce69dfabb4e55b4fd9                  1


In [66]:
from nbresult import ChallengeResult
result = ChallengeResult('number_sellers',
                         shape=Order().get_number_sellers().shape,
                         columns=Order().get_number_sellers().columns)
result.write(); print(result.check())

Constructed CSV Path: /home/vinodha/code/vinodhaminnotra/04-Decision-Science/01-Project-Setup/data-context-and-setup/data/csv
Constructed CSV Path: /home/vinodha/code/vinodhaminnotra/04-Decision-Science/01-Project-Setup/data-context-and-setup/data/csv

============================= test session starts ==============================
platform linux -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /home/vinodha/.pyenv/versions/3.10.6/envs/lewagon/bin/python
cachedir: .pytest_cache
rootdir: /home/vinodha/code/vinodhaminnotra/04-Decision-Science/02-Statistical-Inference/data-orders, configfile: pytest.ini
plugins: asyncio-0.19.0, typeguard-2.13.3, anyio-3.6.2
asyncio: mode=strict
collecting ... collected 2 items

test_number_sellers.py::TestNumberSellers::test_column_names PASSED      [ 50%]
test_number_sellers.py::TestNumberSellers::test_number_seller PASSED     [100%]

============================== 2 passed in 0.40s ===============================


💯 You can commit your code:

git add te

### e) `get_price_and_freight`
     Returns a DataFrame with:
        order_id, price, freight_value

<details>
    <summary>▸ <i>Hint</i></summary>

`pd.Series.agg()` allows you to apply one transformation method per column of your groupby object
</details>

In [67]:
order_items = data['order_items'].copy()
get_price_and_freight_df = order_items.groupby('order_id').agg({
    'price': 'sum',  # Sum the price for each order
    'freight_value': 'sum'  # Sum the freight value for each order
}).reset_index()

print(get_price_and_freight_df.head())

                           order_id   price  freight_value
0  00010242fe8c5a6d1ba2dd792cb16214   58.90          13.29
1  00018f77f2f0320c557190d7a144bdd3  239.90          19.93
2  000229ec398224ef6ca0657da4fc703e  199.00          17.87
3  00024acbcdf0a6daa1e931b038114c75   12.99          12.79
4  00042b26cf59d7ce69dfabb4e55b4fd9  199.90          18.14


In [68]:
from nbresult import ChallengeResult
result = ChallengeResult('price', shape=Order().get_price_and_freight().shape)
result.write(); print(result.check())

Constructed CSV Path: /home/vinodha/code/vinodhaminnotra/04-Decision-Science/01-Project-Setup/data-context-and-setup/data/csv

============================= test session starts ==============================
platform linux -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /home/vinodha/.pyenv/versions/3.10.6/envs/lewagon/bin/python
cachedir: .pytest_cache
rootdir: /home/vinodha/code/vinodhaminnotra/04-Decision-Science/02-Statistical-Inference/data-orders, configfile: pytest.ini
plugins: asyncio-0.19.0, typeguard-2.13.3, anyio-3.6.2
asyncio: mode=strict
collecting ... collected 1 item

test_price.py::TestPrice::test_price PASSED                              [100%]

============================== 1 passed in 0.01s ===============================


💯 You can commit your code:

git add tests/price.pickle

git commit -m 'Completed price step'

git push origin master



## 2. All at once: `get_training_data`

❓ Time to code `get_training_data` making use of your previous coded methods, to gather all order features in one table

In [74]:
# Initialize the Order instance
order_instance = Order()

# Generate the training data
training_data_df = order_instance.get_training_data()

# Display the result
print("Training DataFrame:")
print(training_data_df.head())
print(f"Training DataFrame Shape: {training_data_df.shape}")


Constructed CSV Path: /home/vinodha/code/vinodhaminnotra/04-Decision-Science/01-Project-Setup/data-context-and-setup/data/csv
Training DataFrame:
                           order_id order_status  wait_time  \
0  e481f51cbdc54678b7cc49136f2d6af7    delivered        8.0   
1  53cdb2fc8bc7dce0b6741e2150273451    delivered       13.0   
2  47770eb9100c2d0c44946d9cf07ec65d    delivered        9.0   
3  949d5b44dbf5de918fe9c16f97b45f8a    delivered       13.0   
4  ad21c59c0840e6cb83a9ceb5573f8159    delivered        2.0   

   expected_wait_time  delay_vs_expected  dim_is_five_star  dim_is_one_star  \
0                  15                0.0               0.0              0.0   
1                  19                0.0               0.0              0.0   
2                  26                0.0               1.0              0.0   
3                  26                0.0               1.0              0.0   
4                  12                0.0               1.0              0.0   



In [75]:
# Check each intermediate DataFrame
print(f"Wait Time Shape: {order_instance.get_wait_time().shape}")
print(f"Review Score Shape: {order_instance.get_review_score().shape}")
print(f"Number Items Shape: {order_instance.get_number_items().shape}")
print(f"Number Sellers Shape: {order_instance.get_number_sellers().shape}")
print(f"Price and Freight Shape: {order_instance.get_price_and_freight().shape}")

# Check the shape after each merge
merged_df = order_instance.get_wait_time().merge(order_instance.get_review_score(), on='order_id', how='left')
print(f"After Merging Review Score: {merged_df.shape}")

merged_df = merged_df.merge(order_instance.get_number_items(), on='order_id', how='left')
print(f"After Merging Number Items: {merged_df.shape}")

merged_df = merged_df.merge(order_instance.get_number_sellers(), on='order_id', how='left')
print(f"After Merging Number Sellers: {merged_df.shape}")

merged_df = merged_df.merge(order_instance.get_price_and_freight(), on='order_id', how='left')
print(f"After Merging Price and Freight: {merged_df.shape}")


Wait Time Shape: (96478, 5)
Review Score Shape: (99224, 4)
Number Items Shape: (98666, 2)
Number Sellers Shape: (98666, 2)
Price and Freight Shape: (98666, 3)
After Merging Review Score: (97007, 8)
After Merging Number Items: (97007, 9)
After Merging Number Sellers: (97007, 10)
After Merging Price and Freight: (97007, 12)


🧪  Test it below

In [73]:
from nbresult import ChallengeResult
from olist.order import Order
data = Order().get_training_data()

result = ChallengeResult('training',
    shape=data.shape,
    columns=sorted(list(data.columns))
)
result.write()
print(result.check())

Constructed CSV Path: /home/vinodha/code/vinodhaminnotra/04-Decision-Science/01-Project-Setup/data-context-and-setup/data/csv

============================= test session starts ==============================
platform linux -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /home/vinodha/.pyenv/versions/3.10.6/envs/lewagon/bin/python
cachedir: .pytest_cache
rootdir: /home/vinodha/code/vinodhaminnotra/04-Decision-Science/02-Statistical-Inference/data-orders, configfile: pytest.ini
plugins: asyncio-0.19.0, typeguard-2.13.3, anyio-3.6.2
asyncio: mode=strict
collecting ... collected 2 items

test_training.py::TestTraining::test_training_data_columns PASSED        [ 50%]
test_training.py::TestTraining::test_training_data_shape PASSED          [100%]

============================== 2 passed in 0.01s ===============================


💯 You can commit your code:

git add tests/training.pickle

git commit -m 'Completed training step'

git push origin master



💡 **Not getting the exact number of rows?**
<details><summary>Do you have a difference of 8 rows?</summary>
At the end we want a Dataframe without NaNs. Try to get rid of those. (Don't make it too complex: you only need to change one line in your code.)
</details>
<details><summary>Do you have another difference?</summary>
Maybe something went wrong when merging all the steps in one Dataframe? Check your merges.
</details>

🏁 Congratulations! 

💾 Commit and push your notebook before starting the next challenge.

## 3. [OPTIONAL] `get_distance_seller_customer` 
**Try  to code this function only after finishing today's challenges.**
**First skip to the next challenge.**

    ❓ Returns a Dataframe with:
        order_id, distance_seller_customer (the distance in km between customer and seller)

💡 Have a look at the `haversine_distance` formula we coded for you in the `olist.utils` module

Use this to code `get_distance_seller_customer`.

⚠️ Since an order can have multiple sellers, take the average seller-customer distance per order.

In [72]:
from olist.data import Olist
from olist.utils import haversine_distance

# Load data
olist = Olist()
data = olist.get_data()

# Extract required DataFrames
order_items = data['order_items']
sellers = data['sellers']
customers = data['customers']
geolocation = data['geolocation']
orders = data['orders']

# Verify loaded DataFrames
print(f"Order Items Shape: {order_items.shape}")
print(f"Sellers Shape: {sellers.shape}")
print(f"Customers Shape: {customers.shape}")
print(f"Geolocation Shape: {geolocation.shape}")
print(f"Orders Shape: {orders.shape}")


Constructed CSV Path: /home/vinodha/code/vinodhaminnotra/04-Decision-Science/01-Project-Setup/data-context-and-setup/data/csv
Order Items Shape: (112650, 7)
Sellers Shape: (3095, 4)
Customers Shape: (99441, 5)
Geolocation Shape: (1000163, 5)
Orders Shape: (99441, 8)


In [3]:
# Merge order_items with orders to include customer_id
order_items_with_customer = order_items.merge(
    orders[['order_id', 'customer_id']],
    on='order_id',
    how='left'
)

# Verify the merge
print(f"Order Items with Customer Shape: {order_items_with_customer.shape}")
print(order_items_with_customer.head())


Order Items with Customer Shape: (112650, 8)
                           order_id  order_item_id  \
0  00010242fe8c5a6d1ba2dd792cb16214              1   
1  00018f77f2f0320c557190d7a144bdd3              1   
2  000229ec398224ef6ca0657da4fc703e              1   
3  00024acbcdf0a6daa1e931b038114c75              1   
4  00042b26cf59d7ce69dfabb4e55b4fd9              1   

                         product_id                         seller_id  \
0  4244733e06e7ecb4970a6e2683c13e61  48436dade18ac8b2bce089ec2a041202   
1  e5f2d52b802189ee658865ca93d83a8f  dd7ddc04e1b6c2c614352b383efe2d36   
2  c777355d18b72b67abbeef9df44fd0fd  5b51032eddd242adc84c38acab88f23d   
3  7634da152a4610f1595efa32f14722fc  9d7a1d34a5052409006425275ba1c2b4   
4  ac6c3623068f30de03045865e4e10089  df560393f3a51e74553ab94004ba5c87   

   shipping_limit_date   price  freight_value  \
0  2017-09-19 09:45:35   58.90          13.29   
1  2017-05-03 11:05:13  239.90          19.93   
2  2018-01-18 14:48:30  199.00          17.8

In [4]:
# Merge sellers with geolocation
sellers_geo = sellers.merge(
    geolocation,
    left_on='seller_zip_code_prefix',
    right_on='geolocation_zip_code_prefix',
    how='left'
)

# Verify the merge
print(f"Sellers Geo Shape: {sellers_geo.shape}")
print(sellers_geo.head())


Sellers Geo Shape: (435094, 9)
                          seller_id  seller_zip_code_prefix seller_city  \
0  3442f8959a84dea7ee197c632cb2df15                   13023    campinas   
1  3442f8959a84dea7ee197c632cb2df15                   13023    campinas   
2  3442f8959a84dea7ee197c632cb2df15                   13023    campinas   
3  3442f8959a84dea7ee197c632cb2df15                   13023    campinas   
4  3442f8959a84dea7ee197c632cb2df15                   13023    campinas   

  seller_state  geolocation_zip_code_prefix  geolocation_lat  geolocation_lng  \
0           SP                      13023.0       -22.898536       -47.063125   
1           SP                      13023.0       -22.895499       -47.061944   
2           SP                      13023.0       -22.891740       -47.060820   
3           SP                      13023.0       -22.895762       -47.066144   
4           SP                      13023.0       -22.896154       -47.062431   

  geolocation_city geolocation_

In [5]:
# Merge sellers_geo with order_items_with_customer
order_seller_geo = order_items_with_customer.merge(
    sellers_geo,
    on='seller_id',
    how='left',
    suffixes=('', '_seller')
)

# Verify the result
print(f"Order-Seller Geo Shape: {order_seller_geo.shape}")
print(order_seller_geo.head())


Order-Seller Geo Shape: (16252672, 16)
                           order_id  order_item_id  \
0  00010242fe8c5a6d1ba2dd792cb16214              1   
1  00010242fe8c5a6d1ba2dd792cb16214              1   
2  00010242fe8c5a6d1ba2dd792cb16214              1   
3  00010242fe8c5a6d1ba2dd792cb16214              1   
4  00010242fe8c5a6d1ba2dd792cb16214              1   

                         product_id                         seller_id  \
0  4244733e06e7ecb4970a6e2683c13e61  48436dade18ac8b2bce089ec2a041202   
1  4244733e06e7ecb4970a6e2683c13e61  48436dade18ac8b2bce089ec2a041202   
2  4244733e06e7ecb4970a6e2683c13e61  48436dade18ac8b2bce089ec2a041202   
3  4244733e06e7ecb4970a6e2683c13e61  48436dade18ac8b2bce089ec2a041202   
4  4244733e06e7ecb4970a6e2683c13e61  48436dade18ac8b2bce089ec2a041202   

   shipping_limit_date  price  freight_value  \
0  2017-09-19 09:45:35   58.9          13.29   
1  2017-09-19 09:45:35   58.9          13.29   
2  2017-09-19 09:45:35   58.9          13.29   
3  20

In [6]:
# Merge customers with geolocation
customers_geo = customers.merge(
    geolocation,
    left_on='customer_zip_code_prefix',
    right_on='geolocation_zip_code_prefix',
    how='left'
)

# Verify the merge
print(f"Customers Geo Shape: {customers_geo.shape}")
print(customers_geo.head())


Customers Geo Shape: (15083733, 10)
                        customer_id                customer_unique_id  \
0  06b8999e2fba1a1fbc88172c00ba8bc7  861eff4711a542e4b93843c6dd7febb0   
1  06b8999e2fba1a1fbc88172c00ba8bc7  861eff4711a542e4b93843c6dd7febb0   
2  06b8999e2fba1a1fbc88172c00ba8bc7  861eff4711a542e4b93843c6dd7febb0   
3  06b8999e2fba1a1fbc88172c00ba8bc7  861eff4711a542e4b93843c6dd7febb0   
4  06b8999e2fba1a1fbc88172c00ba8bc7  861eff4711a542e4b93843c6dd7febb0   

   customer_zip_code_prefix customer_city customer_state  \
0                     14409        franca             SP   
1                     14409        franca             SP   
2                     14409        franca             SP   
3                     14409        franca             SP   
4                     14409        franca             SP   

   geolocation_zip_code_prefix  geolocation_lat  geolocation_lng  \
0                      14409.0       -20.509897       -47.397866   
1                      14409

In [2]:
# Merge customers_geo with order_seller_geo
order_seller_customer_geo = order_seller_geo.merge(
    customers_geo,
    on='customer_id',
    how='left',
    suffixes=('', '_customer')
)

# Verify the result
print(f"Order-Seller-Customer Geo Shape: {order_seller_customer_geo.shape}")
print(order_seller_customer_geo.head())


NameError: name 'order_seller_geo' is not defined

In [8]:
matching_geo = Order().get_distance_seller_customer()

NameError: name 'Order' is not defined

👉 Let's quickly check the distribution of the distance seller-customer :)

In [17]:
sns.displot(data = matching_geo, x ='distance_seller_customer')

NameError: name 'sns' is not defined

👉 What is the median and the average seller-customer distance ?

In [ ]:
matching_geo['distance_seller_customer'].describe()

👀 Check your new dataframe and bring your code over to olist/order.py when it works. 

In [ ]:
Order().get_distance_seller_customer()['distance_seller_customer']

🧪  Test your code

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('distance',
    mean = Order().get_distance_seller_customer()['distance_seller_customer'].mean())
result.write()
print(result.check())

Now also update `get_training_data` to merge the `get_distance_seller_customer` if `with_distance_seller_customer` is set to `True`.